Librerias necesarias y credenciales de Foresquare

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.3
  latest version: 4.8.4

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    openssl-1.1.1g             |       h516909a_1         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.2 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.0.0

In [2]:
CLIENT_ID = '10ULFNOQ0LQF41ITBWRDXGFGWZWG5SR3N1HFMPT5FWXV13XM' # your Foursquare ID
CLIENT_SECRET = 'OGQIUGPFEJ4V3OG2B1WYKNLGDK2EI1JK5HDF3YCRNLF155PG' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 10ULFNOQ0LQF41ITBWRDXGFGWZWG5SR3N1HFMPT5FWXV13XM
CLIENT_SECRET:OGQIUGPFEJ4V3OG2B1WYKNLGDK2EI1JK5HDF3YCRNLF155PG


Importar lista de direcciones de los clientes

In [142]:
data_excel = pd.read_excel('direcciones.xlsx', sheet_name='Hoja1')

In [143]:
direcciones = data_excel['direccion'].tolist()

In [144]:
direcciones

['cristobal colon 5672, Las condes',
 'fontanarosa 7108, Las condes',
 'fleming 1200, Las condes',
 'piacenza 1080, Las condes',
 'padre hurtado 300, Las condes',
 'Domingo Bondi 982, Las condes',
 'San Ramon 1796, Las condes',
 'el director 5600, Las condes',
 'alcantara 1026, Las condes',
 'la capitania 120, Las condes',
 'maria teresa 6300, Las condes',
 'pinares 550, Las condes',
 'monroe 8500, Las condes',
 'waterloo 1206, Las condes',
 'tomas moro 1248, Las condes',
 'rio guadiana 9173, Las condes',
 'orion 417, Las condes',
 'Las condes 10373, Las condes',
 'pica 1424, Las condes',
 'chusmiza 1824, Las condes']

Ingresar cada una de las direcciones a foursquare_agent para obtener la Longitud y Latitud

In [193]:
def buscarlonglat(d):
    long_lat = []
    for i in d: 
        geolocator = Nominatim(user_agent="foursquare_agent")
        location = geolocator.geocode(i)
        latitude = location.latitude
        longitude = location.longitude
        print(latitude, longitude)
        long_lat.append([latitude, longitude])
    return long_lat

listalonglat = buscarlonglat(direcciones)

-33.4159953 -70.5371934
-33.41868867142857 -70.55688515714286
-33.4237323 -70.5381523
-33.419338 -70.558986
-33.384372 -70.55005509088694
-33.41706941428571 -70.56823965000001
-33.3986682 -70.5133393
-33.41462562 -70.56927788
-33.4153956 -70.5896875
-33.4126428 -70.5707355
-33.410754757142854 -70.56509321428571
-33.39472997118338 -70.55271020109387
-33.4116519 -70.5431238
-33.40347832914573 -70.54217073417085
-33.4094352 -70.5522029
-33.4124974 -70.5338952
-33.39921474390244 -70.5584674
-33.4084548 -70.5671489
-33.4178416 -70.542695
-23.6305141 -70.3813749


In [194]:
dflonglat = pd.DataFrame(listalonglat)
dflonglat.rename(columns={0:'Latitude',
                        1:'Longitude'},
               inplace=True)
dflonglat

,Latitude,Longitude
0,-33.415995,-70.537193
1,-33.418689,-70.556885
2,-33.423732,-70.538152
3,-33.419338,-70.558986
4,-33.384372,-70.550055
5,-33.417069,-70.568240
6,-33.398668,-70.513339
7,-33.414626,-70.569278
8,-33.415396,-70.589687
9,-33.412643,-70.570735


Se tiene la informacion de cada cliente con su respectiva latitud y longitud obtenida desde Foresquare.

In [195]:
dflonglat['Customer'] = data_excel['Customer']
df_customers = dflonglat[['Customer','Latitude','Longitude']]
df_customers

,Customer,Latitude,Longitude
0,Cliente1,-33.415995,-70.537193
1,Cliente2,-33.418689,-70.556885
2,Cliente3,-33.423732,-70.538152
3,Cliente4,-33.419338,-70.558986
4,Cliente5,-33.384372,-70.550055
5,Cliente6,-33.417069,-70.568240
6,Cliente7,-33.398668,-70.513339
7,Cliente8,-33.414626,-70.569278
8,Cliente9,-33.415396,-70.589687
9,Cliente10,-33.412643,-70.570735


Crear el mapa de Las Condes mostrando la ubicación de cada cliente

In [196]:
# create map of Las Condes, Santiago, Chile using latitude and longitude values
map_lascondes = folium.Map(location=[-33.412406 , -70.549878], zoom_start=13)

# add markers to map
for lat, lng, label in zip(df_customers['Latitude'], df_customers['Longitude'], df_customers['Customer']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_lascondes)  
    
map_lascondes

Se realizará el ejercicio para 1 solo cliente a modo de prueba. En este caso será el cliente2

In [197]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            lat, 
            lng, 
            VERSION, 
            search_query, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()['response']['venues']  #["response"]['groups'][0]['items']
       
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng,
            v['name'],
            v['location']['lat'],
            v['location']['lng']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Customer', 
                  'Customer Latitude', 
                  'Customer Longitude', 
                  'Cafe', 
                  'Cafe Latitude', 
                  'Cafe Longitude']
    
    return(nearby_venues)

In [198]:
lascondes_venues = getNearbyVenues(names=df_customers['Customer'],
                                   latitudes=df_customers['Latitude'],
                                   longitudes=df_customers['Longitude']
                                  )

In [199]:
lascondes_venues

,Customer,Customer Latitude,Customer Longitude,Cafe,Cafe Latitude,Cafe Longitude
0,Cliente1,-33.415995,-70.537193,Work Café Santander,-33.416046,-70.539345
1,Cliente1,-33.415995,-70.537193,Juan Valdez café,-33.415450,-70.539978
2,Cliente2,-33.418689,-70.556885,MosaiCafé,-33.417324,-70.553928
3,Cliente2,-33.418689,-70.556885,Café Marliz,-33.417479,-70.558567
4,Cliente2,-33.418689,-70.556885,McCafé,-33.417408,-70.553184
5,Cliente2,-33.418689,-70.556885,Cafe Inti,-33.419464,-70.551691
6,Cliente2,-33.418689,-70.556885,Café Tomás,-33.419195,-70.551579
7,Cliente2,-33.418689,-70.556885,Inti Cafe Boutique,-33.419327,-70.551526
8,Cliente2,-33.418689,-70.556885,Dolce Capriccio Cafetería,-33.418848,-70.551705
9,Cliente3,-33.423732,-70.538152,El Ombú Café,-33.423330,-70.538529


In [204]:
conteo_frecuencia = lascondes_venues.groupby('Customer').count().reset_index()
df1 = conteo_frecuencia[['Customer', 'Cafe']]
df1.columns = ['Customer', 'FreqCoffe']
df1

,Customer,FreqCoffe
0,Cliente1,2
1,Cliente10,22
2,Cliente11,19
3,Cliente12,2
4,Cliente13,1
5,Cliente14,2
6,Cliente15,3
7,Cliente16,1
8,Cliente17,1
9,Cliente18,26


Unir df_customers y df1

In [205]:
dffinal = pd.merge(df_customers, df1, on='Customer')

Cantidad de cafés cercanos en un radio de 500 metros

In [206]:
dffinal

,Customer,Latitude,Longitude,FreqCoffe
0,Cliente1,-33.415995,-70.537193,2
1,Cliente2,-33.418689,-70.556885,7
2,Cliente3,-33.423732,-70.538152,3
3,Cliente4,-33.419338,-70.558986,3
4,Cliente5,-33.384372,-70.550055,3
5,Cliente6,-33.417069,-70.568240,7
6,Cliente7,-33.398668,-70.513339,6
7,Cliente8,-33.414626,-70.569278,18
8,Cliente9,-33.415396,-70.589687,26
9,Cliente10,-33.412643,-70.570735,22


Clusterizar

In [212]:
# set number of clusters
kclusters = 3

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:20] 

array([0, 2, 0, 0, 0, 2, 2, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0],
      dtype=int32)

In [213]:
# add clustering labels
dffinal.insert(4, 'Cluster Labels', kmeans.labels_)

dffinal

,Customer,Latitude,Longitude,FreqCoffe,Cluster Labels
0,Cliente1,-33.415995,-70.537193,2,0
1,Cliente2,-33.418689,-70.556885,7,2
2,Cliente3,-33.423732,-70.538152,3,0
3,Cliente4,-33.419338,-70.558986,3,0
4,Cliente5,-33.384372,-70.550055,3,0
5,Cliente6,-33.417069,-70.568240,7,2
6,Cliente7,-33.398668,-70.513339,6,2
7,Cliente8,-33.414626,-70.569278,18,1
8,Cliente9,-33.415396,-70.589687,26,1
9,Cliente10,-33.412643,-70.570735,22,1


Crear el mapa

In [218]:
# create map
map_clusters = folium.Map(location=[-33.412406 , -70.549878], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, customer, cluster, frec in zip(dffinal['Latitude'], dffinal['Longitude'], dffinal['Customer'], dffinal['Cluster Labels'], dffinal['FreqCoffe']):
    label = folium.Popup(str(customer) + ' - Cluster' + str(cluster) + ' - FreqCoffe = ' +str(frec)  , parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [219]:
dffinalorde = dffinal.sort_values('FreqCoffe')
dffinalorde

,Customer,Latitude,Longitude,FreqCoffe,Cluster Labels
19,Cliente20,-23.630514,-70.381375,1,0
16,Cliente17,-33.399215,-70.558467,1,0
15,Cliente16,-33.412497,-70.533895,1,0
12,Cliente13,-33.411652,-70.543124,1,0
13,Cliente14,-33.403478,-70.542171,2,0
11,Cliente12,-33.394730,-70.552710,2,0
18,Cliente19,-33.417842,-70.542695,2,0
0,Cliente1,-33.415995,-70.537193,2,0
4,Cliente5,-33.384372,-70.550055,3,0
3,Cliente4,-33.419338,-70.558986,3,0
